In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
import tensorflow.keras.layers as L
import pandas as pd
import numpy as np

In [2]:
IMAGE_DIR = '../input/origalist/Images/Images'

In [3]:
df = pd.read_csv('../input/origalist/OrigaList.csv')
df.drop(['Eye', 'ExpCDR', 'Set'], axis=1, inplace=True)
df.head()

,Filename,Glaucoma
0,001.jpg,0
1,002.jpg,0
2,003.jpg,0
3,004.jpg,0
4,005.jpg,0


In [4]:
data_gen = image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.3,
                                   rotation_range=20,
                                   validation_split=0.1,
                                   width_shift_range=0.3)

train_generator = data_gen.flow_from_dataframe(df,
                                              IMAGE_DIR,
                                              x_col='Filename',
                                              y_col='Glaucoma',
                                              target_size=(256, 256),
                                              class_mode='raw',
                                              batch_size=16,
                                              shuffle=True,
                                              subset='training')

val_generator = data_gen.flow_from_dataframe(df,
                                            IMAGE_DIR,
                                            x_col='Filename',
                                            y_col='Glaucoma',
                                            target_size=(256, 256),
                                            class_mode='raw',
                                            batch_size=8,
                                            subset='validation')

Found 585 validated image filenames.
Found 65 validated image filenames.


In [5]:
sum(df['Glaucoma'])/len(df['Glaucoma'])

0.25846153846153846

In [8]:
model = keras.Sequential()
model.add(InceptionResNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet'))
model.add(L.GlobalAveragePooling2D())
model.add(L.Dropout(0.4))
model.add(L.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', keras.metrics.AUC()])

In [9]:
history = model.fit(train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size,
                   validation_data=val_generator, validation_steps=val_generator.n//val_generator.batch_size,
                   epochs=8)

Epoch 1/8
36/36 [==============================] - 61s 2s/step - loss: 0.7191 - acc: 0.6819 - auc_1: 0.5945 - val_loss: 2000.8706 - val_acc: 0.0625 - val_auc_1: 0.5000
Epoch 2/8
36/36 [==============================] - 56s 2s/step - loss: 0.5456 - acc: 0.7241 - auc_1: 0.7195 - val_loss: 2.8806 - val_acc: 0.9375 - val_auc_1: 0.6542
Epoch 3/8
36/36 [==============================] - 58s 2s/step - loss: 0.5585 - acc: 0.7381 - auc_1: 0.7135 - val_loss: 2443.8853 - val_acc: 0.9062 - val_auc_1: 0.3417
Epoch 4/8
36/36 [==============================] - 54s 2s/step - loss: 0.5429 - acc: 0.7292 - auc_1: 0.7189 - val_loss: 0.5761 - val_acc: 0.9062 - val_auc_1: 0.5355
Epoch 5/8
36/36 [==============================] - 55s 2s/step - loss: 0.5248 - acc: 0.7364 - auc_1: 0.7421 - val_loss: 0.2934 - val_acc: 0.8594 - val_auc_1: 0.7875
Epoch 6/8
36/36 [==============================] - 54s 2s/step - loss: 0.4786 - acc: 0.7873 - auc_1: 0.8013 - val_loss: 0.2240 - val_acc: 0.9375 - val_auc_1: 0.8917
Epoc